In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
train_df

,id,gravity,ph,osmo,cond,urea,calc,target
0,0,1.013,6.19,443,14.8,124,1.45,0
1,1,1.025,5.40,703,23.6,394,4.18,0
2,2,1.009,6.13,371,24.5,159,9.04,0
3,3,1.021,4.91,442,20.8,398,6.63,1
4,4,1.021,5.53,874,17.8,385,2.21,1
...,...,...,...,...,...,...,...,...
409,409,1.011,5.21,527,21.4,75,1.53,0
410,410,1.024,5.53,577,19.7,224,0.77,0
411,411,1.018,6.28,455,22.2,270,7.68,1
412,412,1.008,7.12,325,12.6,75,1.03,1


In [ ]:
X = train_df.drop(columns=['target','id'])  # Features
y = train_df['target'] #target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print("Shape of input features (X_train):", X_train.shape)
print("Shape of target variable (y_train):", y_train.shape)

Shape of input features (X_train): (331, 6)
Shape of target variable (y_train): (331,)


## Base model

In [ ]:
input_shape = (X.shape[1],)
input_shape

(6,)

In [ ]:
model = None
model = Sequential()
model.add(Dense(6, activation='relu', input_shape=input_shape))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=25, validation_split=0.2)

Epoch 1/25
9/9 [==============================] - 3s 68ms/step - loss: 238.5737 - accuracy: 0.4394 - val_loss: 238.1977 - val_accuracy: 0.4478
Epoch 2/25
9/9 [==============================] - 0s 13ms/step - loss: 228.3121 - accuracy: 0.4394 - val_loss: 227.8537 - val_accuracy: 0.4478
Epoch 3/25
9/9 [==============================] - 0s 15ms/step - loss: 218.3023 - accuracy: 0.4394 - val_loss: 217.7660 - val_accuracy: 0.4478
Epoch 4/25
9/9 [==============================] - 0s 11ms/step - loss: 208.8077 - accuracy: 0.4394 - val_loss: 208.0436 - val_accuracy: 0.4478
Epoch 5/25
9/9 [==============================] - 0s 20ms/step - loss: 199.2580 - accuracy: 0.4394 - val_loss: 198.8426 - val_accuracy: 0.4478
Epoch 6/25
9/9 [==============================] - 0s 21ms/step - loss: 190.3252 - accuracy: 0.4394 - val_loss: 189.8024 - val_accuracy: 0.4478
Epoch 7/25
9/9 [==============================] - 0s 12ms/step - loss: 181.7715 - accuracy: 0.4394 - val_loss: 180.7142 - val_accuracy: 0.4478

In [ ]:
val_loss, val_accuracy = model.evaluate(X_test, y_test)

3/3 [==============================] - 0s 7ms/step - loss: 56.2143 - accuracy: 0.4578


In [ ]:
print("Validation Accuracy:", val_accuracy)

Validation Accuracy: 0.45783132314682007


---
## Preparing data

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from imblearn.over_sampling import SMOTE
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam, SGD

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
x = train_df.drop(['id', 'target'], axis=1)
y = train_df['target']

In [ ]:
y.value_counts()

target
0    230
1    184
Name: count, dtype: int64

In [ ]:
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(x,y)

In [ ]:
y_res.value_counts()

target
0    230
1    230
Name: count, dtype: int64

In [ ]:
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X_res)
print(X_poly)

[[1.00000000e+00 1.01300000e+00 6.19000000e+00 ... 1.53760000e+04
  1.79800000e+02 2.10250000e+00]
 [1.00000000e+00 1.02500000e+00 5.40000000e+00 ... 1.55236000e+05
  1.64692000e+03 1.74724000e+01]
 [1.00000000e+00 1.00900000e+00 6.13000000e+00 ... 2.52810000e+04
  1.43736000e+03 8.17216000e+01]
 ...
 [1.00000000e+00 1.02016527e+00 5.45274446e+00 ... 2.61121000e+05
  3.58080417e+03 4.91042794e+01]
 [1.00000000e+00 1.02487491e+00 5.68297092e+00 ... 1.48225000e+05
  1.53537021e+03 1.59039412e+01]
 [1.00000000e+00 1.02791918e+00 5.43604350e+00 ... 1.71396000e+05
  4.35673792e+03 1.10744506e+02]]


In [ ]:
scaler = StandardScaler()
X_val_scaled = scaler.fit_transform(X_poly)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_val_scaled, y_res, test_size=0.2, random_state=42)

In [ ]:
print("Shape of input features (X_train):", X_train.shape)
print("Shape of target variable (y_train):", y_train.shape)

Shape of input features (X_train): (368, 28)
Shape of target variable (y_train): (368,)


# **Model 2**

In [ ]:
#

In [ ]:
input_shape = (X_train.shape[1],)

In [ ]:
model = None
model = Sequential()
model.add(Dense(6, activation='relu', input_shape=input_shape))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
adam=Adam(learning_rate=0.001)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=25, validation_split=0.2)

Epoch 1/25
10/10 [==============================] - 2s 71ms/step - loss: 0.8014 - accuracy: 0.5068 - val_loss: 0.7463 - val_accuracy: 0.5135
Epoch 2/25
10/10 [==============================] - 0s 20ms/step - loss: 0.7565 - accuracy: 0.5000 - val_loss: 0.7243 - val_accuracy: 0.5270
Epoch 3/25
10/10 [==============================] - 0s 15ms/step - loss: 0.7190 - accuracy: 0.5170 - val_loss: 0.7071 - val_accuracy: 0.5541
Epoch 4/25
10/10 [==============================] - 0s 24ms/step - loss: 0.6885 - accuracy: 0.5374 - val_loss: 0.6943 - val_accuracy: 0.5541
Epoch 5/25
10/10 [==============================] - 0s 9ms/step - loss: 0.6658 - accuracy: 0.6020 - val_loss: 0.6852 - val_accuracy: 0.5946
Epoch 6/25
10/10 [==============================] - 0s 9ms/step - loss: 0.6512 - accuracy: 0.6224 - val_loss: 0.6756 - val_accuracy: 0.5811
Epoch 7/25
10/10 [==============================] - 0s 12ms/step - loss: 0.6380 - accuracy: 0.6327 - val_loss: 0.6686 - val_accuracy: 0.6081
Epoch 8/25
10/1

In [ ]:
val_loss, val_accuracy = model.evaluate(X_test, y_test)

3/3 [==============================] - 0s 7ms/step - loss: 0.6664 - accuracy: 0.6522


In [ ]:
print("Validation Accuracy:", val_accuracy)

Validation Accuracy: 0.6521739363670349


---

# **Model 3**

In [ ]:
model = None
model = Sequential()
model.add(Dense(6, activation='relu', input_shape=input_shape))
model.add(Dense(9, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
adam=Adam(learning_rate=0.001)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=25, validation_split=0.2)

Epoch 1/25
10/10 [==============================] - 1s 24ms/step - loss: 0.6993 - accuracy: 0.5034 - val_loss: 0.6911 - val_accuracy: 0.5270
Epoch 2/25
10/10 [==============================] - 0s 7ms/step - loss: 0.6769 - accuracy: 0.5714 - val_loss: 0.6844 - val_accuracy: 0.6081
Epoch 3/25
10/10 [==============================] - 0s 7ms/step - loss: 0.6622 - accuracy: 0.6497 - val_loss: 0.6828 - val_accuracy: 0.5811
Epoch 4/25
10/10 [==============================] - 0s 7ms/step - loss: 0.6513 - accuracy: 0.6735 - val_loss: 0.6824 - val_accuracy: 0.5946
Epoch 5/25
10/10 [==============================] - 0s 7ms/step - loss: 0.6404 - accuracy: 0.7177 - val_loss: 0.6815 - val_accuracy: 0.5811
Epoch 6/25
10/10 [==============================] - 0s 5ms/step - loss: 0.6318 - accuracy: 0.7381 - val_loss: 0.6809 - val_accuracy: 0.6216
Epoch 7/25
10/10 [==============================] - 0s 6ms/step - loss: 0.6237 - accuracy: 0.7449 - val_loss: 0.6804 - val_accuracy: 0.6216
Epoch 8/25
10/10 [=

In [ ]:
val_loss, val_accuracy = model.evaluate(X_test, y_test)

3/3 [==============================] - 0s 4ms/step - loss: 0.6766 - accuracy: 0.6848


In [ ]:
print("Validation Accuracy:", val_accuracy)

Validation Accuracy: 0.6847826242446899


---

# **Model 4n**

In [ ]:
input_shape

(28,)

In [ ]:
model = None
model = Sequential()
model.add(Dropout(0.1, input_shape=input_shape))
model.add(Dense(28, activation='relu'))
model.add(Dense(56, activation='relu'))
model.add(Dense(14, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
adam=Adam(learning_rate=0.001)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=3, factor=0.1)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=7)

history = model.fit(X_train, y_train, epochs=30, validation_split=0.2, callbacks=[learning_rate_reduction, early_stopping])
#history = model.fit(X_train, y_train, epochs=25, validation_split=0.2)

Epoch 1/30
10/10 [==============================] - 1s 26ms/step - loss: 0.6380 - accuracy: 0.6190 - val_loss: 0.6980 - val_accuracy: 0.5676 - lr: 0.0010
Epoch 2/30
10/10 [==============================] - 0s 7ms/step - loss: 0.5952 - accuracy: 0.7041 - val_loss: 0.7098 - val_accuracy: 0.5811 - lr: 0.0010
Epoch 3/30
10/10 [==============================] - 0s 7ms/step - loss: 0.5770 - accuracy: 0.6905 - val_loss: 0.7149 - val_accuracy: 0.6081 - lr: 0.0010
Epoch 4/30
10/10 [==============================] - 0s 6ms/step - loss: 0.5671 - accuracy: 0.7177 - val_loss: 0.7164 - val_accuracy: 0.6081 - lr: 0.0010
Epoch 5/30
10/10 [==============================] - 0s 8ms/step - loss: 0.5603 - accuracy: 0.7381 - val_loss: 0.7156 - val_accuracy: 0.5946 - lr: 0.0010
Epoch 6/30
10/10 [==============================] - 0s 6ms/step - loss: 0.5354 - accuracy: 0.7619 - val_loss: 0.7289 - val_accuracy: 0.6081 - lr: 0.0010
Epoch 7/30
10/10 [==============================] - 0s 7ms/step - loss: 0.5342 - 

In [ ]:
val_loss, val_accuracy = model.evaluate(X_test, y_test)

3/3 [==============================] - 0s 5ms/step - loss: 0.7089 - accuracy: 0.6957


In [ ]:
print("Validation Accuracy:", val_accuracy)

Validation Accuracy: 0.695652186870575


# **Model 5**

---

In [ ]:
model = None
model = Sequential()
model.add(Dense(6, activation='relu', input_shape=input_shape))
Dropout(0.5)
model.add(Dense(6, activation='relu'))
Dropout(0.5)
model.add(Dense(2, activation='softmax'))

In [ ]:
adam=Adam(learning_rate=0.001)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=3, factor=0.1)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=7)

history = model.fit(X_train, y_train, epochs=30, validation_split=0.2, callbacks=[learning_rate_reduction, early_stopping])

Epoch 1/30
10/10 [==============================] - 1s 23ms/step - loss: 0.6855 - accuracy: 0.5374 - val_loss: 0.7484 - val_accuracy: 0.4730 - lr: 0.0010
Epoch 2/30
10/10 [==============================] - 0s 7ms/step - loss: 0.6657 - accuracy: 0.5884 - val_loss: 0.7367 - val_accuracy: 0.5541 - lr: 0.0010
Epoch 3/30
10/10 [==============================] - 0s 7ms/step - loss: 0.6526 - accuracy: 0.6293 - val_loss: 0.7299 - val_accuracy: 0.5811 - lr: 0.0010
Epoch 4/30
10/10 [==============================] - 0s 7ms/step - loss: 0.6436 - accuracy: 0.6701 - val_loss: 0.7292 - val_accuracy: 0.5541 - lr: 0.0010
Epoch 5/30
10/10 [==============================] - 0s 6ms/step - loss: 0.6373 - accuracy: 0.6871 - val_loss: 0.7301 - val_accuracy: 0.5676 - lr: 0.0010
Epoch 6/30
10/10 [==============================] - 0s 7ms/step - loss: 0.6318 - accuracy: 0.7109 - val_loss: 0.7294 - val_accuracy: 0.5811 - lr: 0.0010
Epoch 7/30
10/10 [==============================] - 0s 8ms/step - loss: 0.6287 - 

In [ ]:
val_loss, val_accuracy = model.evaluate(X_test, y_test)

3/3 [==============================] - 0s 4ms/step - loss: 0.7257 - accuracy: 0.6087


In [ ]:
print("Validation Accuracy:", val_accuracy)

Validation Accuracy: 0.6086956262588501
